In [1]:
import warnings
import os
import io
import IPython.display as display
from PIL import Image
import base64
from dotenv import load_dotenv, find_dotenv
from huggingface_hub import login
import pandas as pd
import numpy as np
from smolagents import tool

warnings.filterwarnings("ignore")
_ = load_dotenv(find_dotenv())
login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


In [7]:
def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost


suppliers_df["daily_price"] = suppliers_df.apply(calculate_daily_supplier_price, axis=1)
display.display(suppliers_df)

# Let's remove this column now.
suppliers_df = suppliers_df.drop("daily_price", axis=1)

,name,location,distance_km,canadian,price_per_liter,tasting_fee,daily_price
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00,206.175663
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50,171.800000
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14,568.170619
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00,378.900000
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20,406.100000


In [10]:
@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed


@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0

In [11]:
calculate_transport_cost.description

'Calculate transportation cost based on distance and order size.\nRefrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.'

### SETUP THE MODEL

In [ ]:
from smolagents import HfApiModel, CodeAgent
# from helper import get_huggingface_token

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together",  # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1,
)

### SETUP THE CODE AGENT

In [16]:
agent = CodeAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2,
)
agent.logger.console.width = 66

In [17]:
agent.run(
    """Can you get me the transportation cost for 50 liters
    of ice cream over 10 kilometers?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the transportation cost for 50 liters           │
│     of ice cream over 10 kilometers?                           │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` function to    
calculate the cost of transporting 50 liters of ice cream over 10 
kilometers.                                                       
Code:                                                             
```py                                                             
distance_km = 10                                                  
order_volume = 50                                                 
transport_cost = calculate_transport_cost(distance_km,            
order_volume)                                                     
print(f"The transportation cost for 50 liters of ice cream over 10
kilometers is {transport_cost}.")                                 
```                                                               

─ Executing parsed code: ─────────────────────────────────────── 
  distance_km = 10                                                
  order_volume = 50                                               
  transport_cost = calculate_transport_cost(distance_km,          
  order_volume)                                                   
  print(f"The transportation cost for 50 liters of ice cream      
  over 10 kilometers is {transport_cost}.")                       
 ────────────────────────────────────────────────────────────────

Execution logs:
The transportation cost for 50 liters of ice cream over 10 
kilometers is 12.0.

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: I have the transportation cost for 50 liters of ice cream
over 10 kilometers. I will now provide the final answer.          
Code:                                                             
```py                                                             
final_answer(12.0)                                                
```                                                               

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.0)                                              
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 2.88 seconds| Input tokens: 4,601 | Output 
tokens: 138]

12.0

#### GIVE DETAILED INSTRUCTION TO AGENT

In [18]:
task = """Here is a dataframe of different ice cream suppliers.
Could you give me a comparative table (as a dataframe) of the total
daily price for getting daily ice cream delivery from each of them,
given that we need exactly 30 liters of ice cream per day? Take
into account transportation cost and tariffs.
"""
agent.logger.level = 1
agent.run(
    task,
    additional_args={
        "suppliers_data": suppliers_df,
        "data_description": data_description,
    },
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Here is a dataframe of different ice cream suppliers.          │
│ Could you give me a comparative table (as a dataframe) of the  │
│ total                                                          │
│ daily price for getting daily ice cream delivery from each of  │
│ them,                                                          │
│ given that we need exactly 30 liters of ice cream per day?     │
│ Take                                                           │
│ into account transportation cost and tariffs.                  │
│                                                                │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  import pandas as pd                                             
                                                                  
  # Load the data                                                 
  data = suppliers_data                                           
                                                                  
  # Calculate the total cost for each supplier                    
  def calculate_total_cost(row):                                  
      # Calculate the cost of the ice cream                       
      ice_cream_cost = row['price_per_liter'] * 30                
      # Calculate the transportation cost                         
      transport_cost =                                            
  calculate_transport_cost(row['distance_km'], 30)                
      # Calculate the tariff                                      
      tariff = calculate_tariff(ice_cream_cost + transport_cost,  
  row['canadian'])                                                
      # Calculate the total cost                                  
      total_cost = ice_cream_cost + transport_cost + tariff +     
  row['tasting_fee']                                              
      return total_cost                                           
                                                                  
  # Apply the function to each row                                
  data['total_daily_cost'] = data.apply(calculate_total_cost,     
  axis=1)                                                         
                                                                  
  # Create a new dataframe with the total daily cost for each     
  supplier                                                        
  result = data[['name', 'total_daily_cost']]                     
  print(result)                                                   
 ────────────────────────────────────────────────────────────────

Execution logs:
                    name  total_daily_cost
0  Montreal Ice Cream Co        215.223450
1  Brain Freeze Brothers        171.800000
2     Toronto Gelato Ltd        598.329909
3         Buffalo Scoops        378.900000
4       Vermont Creamery        406.100000

Out: None

[Step 1: Duration 8.33 seconds| Input tokens: 2,466 | Output 
tokens: 249]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(result)                                            
 ────────────────────────────────────────────────────────────────

Out - Final answer:                     name  total_daily_cost
0  Montreal Ice Cream Co        215.223450
1  Brain Freeze Brothers        171.800000
2     Toronto Gelato Ltd        598.329909
3         Buffalo Scoops        378.900000
4       Vermont Creamery        406.100000

[Step 2: Duration 3.10 seconds| Input tokens: 5,544 | Output 
tokens: 293]

,name,total_daily_cost
0,Montreal Ice Cream Co,215.223450
1,Brain Freeze Brothers,171.800000
2,Toronto Gelato Ltd,598.329909
3,Buffalo Scoops,378.900000
4,Vermont Creamery,406.100000


#### TRADITIONAL TOOL CALLING
Thought: I need to use the `calculate_transport_cost` tool
to get the transportation cost for 50 liters of ice cream over
10 kilometers.
                        
Code:
```py                                  
transport_cost = calculate_transport_cost(
    distance_km=10,
    order_volume=50
)
print(transport_cost)                             
```
Thought: I need to use the `calculate_transport_cost` tool
to get the transportation cost for 50 liters of ice cream over
10 kilometers.

Action:
{
    "tool_name": "calculate_transport_cost",
    "tool_arguments": {
        "distance_km": 10,
        "order_volume": 50
    }
}

In [ ]:
from smolagents import ToolCallingAgent
model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    temperature=0.6,
)
agent = ToolCallingAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=20,
)
agent.logger.console.width = 66

In [ ]:
# THIS MAY THROW AN ERROR
output = agent.run(
    task,
    additional_args={
        "suppliers_data": suppliers_df,
        "data_description": data_description,
    },
)
print(output)